In [4]:
import pandas as pd
import numpy as np
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

In [5]:
# get column yang dibutuhkan
df = pd.read_csv('data/data-mentah.csv', usecols=["tweet_id", "author_location", "latitude", "longitude", "address", "country", "province", "city"])

# check location bukan null atau empty
df = df[df['author_location'].isnull() == False]
df = df[df['author_location'].isna() == False]

df

,tweet_id,author_location,address,longitude,latitude,province,city,country
0,153619597713608704,di dalam asa pemuda,NaN,NaN,NaN,NaN,NaN,NaN
1,153857491925610496,"Bogor, Indonesia","Bogor, West Java, 16121, Indonesia",106.797242,-6.596299,Jawa Barat,Kelurahan Pabaton,Indonesia
2,154361424154603520,Indonesia,Indonesia,117.890285,-2.483383,Kalimantan Timur,NaN,Indonesia
3,156397219346518017,"ÜT: -6.895087,107.627799",NaN,NaN,NaN,NaN,NaN,NaN
4,156936339718279168,Indonesia,Indonesia,117.890285,-2.483383,Kalimantan Timur,NaN,Indonesia
...,...,...,...,...,...,...,...,...
109697,1542505510510817287,"Jakarta, Indonesia","Jakarta Special Capital Region, Indonesia",106.827183,-6.175394,DKI Jakarta,Daerah Khusus Ibukota Jakarta,Indonesia
109699,1542524597559775232,Bandarlampung - Indonesia,NaN,NaN,NaN,NaN,NaN,NaN
109701,1542540419677859842,Bandarlampung - Indonesia,NaN,NaN,NaN,NaN,NaN,NaN
109703,1542592685978320896,Sumba Tengah,"Sumba Tengah, East Nusa Tenggara, Indonesia",119.666277,-9.593584,Nusa Tenggara Timur,NaN,Indonesia


In [4]:
# function yang mengubah text location menjadi longitude dan latitude
def lookup_location(location):
    # initialize
    geo_locator = Nominatim(user_agent="LearnPython")

    try:
        # get latitude
        coordinate = geo_locator.geocode(location, language='en')
    except GeocoderTimedOut:
        # if timeout return none
        return None
    except:
        # selain itu none
        return None
    
    return coordinate

In [5]:
# function untuk mendapatkan lokasi name (country, province, city) berdasarkan latitude dan longitude
def lookup_location_by_coordinate(latitude, longitude):
    # initialize
    geolocator = Nominatim(user_agent="geoapiExercises")

    try:
        # get location name
        location = geolocator.reverse([latitude+","+longitude])
    except GeocoderTimedOut:
        return None
    except:
        return None
    
    return location

In [6]:
# load semua data yang memiliki author location
for index, row in df.iterrows():
    # cek author location ada / tidak kosong
    if row['author_location']:
        # calling function get coordinate
        coordinate = lookup_location(row['author_location'])
        if coordinate is not None:
            # isi row berdasarkan data
            df.loc[index, 'address'] = coordinate.address
            df.loc[index, 'latitude'] = coordinate.latitude
            df.loc[index, 'longitude'] = coordinate.longitude

            # print("address "+ coordinate.address)
            # print("latitude "+ str(coordinate.latitude))
            # print("longitude "+ str(coordinate.longitude))

            # calling function get location detail
            location_detail = lookup_location_by_coordinate(coordinate.latitude,coordinate.longitude )
            if location_detail:
                address = location_detail.raw['address']
                country = address.get('country', '')
                province = address.get('state', '')
                city = address.get('city', '')
                
                if country:
                    # isi row berdasarkan data
                    df.loc[index, 'country'] = country
                    df.loc[index, 'province'] = province
                    df.loc[index, 'city'] = city

                    # print("country "+ country)
                    # print("province "+ province)
                    # print("city "+ city)
        else:
            df.loc[index, 'address'] = ""

In [ ]:
# save data geocoding
df.to_csv('data/data-geocoding.csv')